Import the Q# module.

This enables the `%%qsharp` magic and initializes a Q# interpreter singleton.

In [8]:
import qsharp


Run Q# using the `%%qsharp` magic.

`DumpMachine()` and `Message()` output get formatted as HTML. Return value is shown as cell output.

In [9]:
%%qsharp

operation Main() : Result {
    use q = Qubit();
    X(q);
    Microsoft.Quantum.Diagnostics.DumpMachine();
    let r = M(q);
    Message($"The result of the measurement is {r}");
    Reset(q);
    r
}

Main()


STATE:
|111⟩: 1.0000+0.0000𝑖

The result of the measurement is One

One

`qsharp.eval()` does the same thing as the `%%qsharp` magic.

`DumpMachine()` and `Message()` print to stdout and get displayed in the notebook as plain text

In [10]:
qsharp.eval("Main()")


STATE:
|111⟩: 1.0000+0.0000𝑖
The result of the measurement is One


One

Assign a result to a Python variable.

In [11]:
result = qsharp.eval("1 + 2")

print(f"Result: {result} (type: {type(result).__name__})")


Result: 3 (type: int)


Errors are exceptions. 

Catch and handle compilation errors.

In [12]:
from qsharp import QSharpError

try:
    qsharp.eval(
        """
operation Foo() : Unit {
    Bar();
    Baz();
}
"""
    )
except QSharpError as ex:
    print(ex)


Qsc.Resolve.NotFound

  × name error
  ╰─▶ `Bar` not found
   ╭─[line_9:2:1]
 2 │ operation Foo() : Unit {
 3 │     Bar();
   ·     ───
 4 │     Baz();
   ╰────
Qsc.Resolve.NotFound

  × name error
  ╰─▶ `Baz` not found
   ╭─[line_9:3:1]
 3 │     Bar();
 4 │     Baz();
   ·     ───
 5 │ }
   ╰────
Qsc.TypeCk.AmbiguousTy

  × type error
  ╰─▶ insufficient type information to infer type
   ╭─[line_9:2:1]
 2 │ operation Foo() : Unit {
 3 │     Bar();
   ·     ─────
 4 │     Baz();
   ╰────
  help: provide a type annotation
Qsc.TypeCk.AmbiguousTy

  × type error
  ╰─▶ insufficient type information to infer type
   ╭─[line_9:3:1]
 3 │     Bar();
 4 │     Baz();
   ·     ─────
 5 │ }
   ╰────
  help: provide a type annotation



Catch and handle runtime errors.

In [13]:
try:
    qsharp.eval("operation Foo() : Unit { use q = Qubit(); X(q) } Foo()")
except QSharpError as ex:
    print(ex)


Error: Qubit2 released while not in |0⟩ state
Call stack:
    at QIR.Runtime.__quantum__rt__qubit_release in core/qir.qs
    at Foo in line_10
Qsc.Eval.ReleasedQubitNotZero

  × runtime error
  ╰─▶ Qubit2 released while not in |0⟩ state
   ╭─[line_10:1:1]
 1 │ operation Foo() : Unit { use q = Qubit(); X(q) } Foo()
   ·                          ────────┬───────
   ·                                  ╰── Qubit2
   ╰────
  help: qubits should be returned to the |0⟩ state before being released to
        satisfy the assumption that allocated qubits start in the |0⟩ state



A runtime error that's not caught gets reported as a Python exception.

In [14]:
qsharp.eval("operation Foo() : Unit { use q = Qubit(); X(q) } Foo()")


QSharpError: Error: Qubit3 released while not in |0⟩ state
Call stack:
    at QIR.Runtime.__quantum__rt__qubit_release in core/qir.qs
    at Foo in line_11
[31mQsc.Eval.ReleasedQubitNotZero[0m

  [31m×[0m runtime error
[31m  ╰─▶ [0mQubit3 released while not in |0⟩ state
   ╭─[[36;1;4mline_11[0m:1:1]
 [2m1[0m │ operation Foo() : Unit { use q = Qubit(); X(q) } Foo()
   · [35;1m                         ────────┬───────[0m
   ·                                  [35;1m╰── [35;1mQubit3[0m[0m
   ╰────
[36m  help: [0mqubits should be returned to the |0⟩ state before being released to
        satisfy the assumption that allocated qubits start in the |0⟩ state


In `%%qsharp` cells, exceptions are handled and displayed as error text.

In [ ]:
%%qsharp

operation Bar() : Unit {
    use q = Qubit();
    Microsoft.Quantum.Diagnostics.DumpMachine();
    X(q);
}

Bar()


Streaming output for long running operations.

In [ ]:
%%qsharp

open Microsoft.Quantum.Diagnostics;

operation Main() : Unit {
    Message("Generating random bit... ");
    for i in 0..400000 {
        use q = Qubit();
        H(q);
        let r = M(q);
        if (i % 100000) == 0 {
            DumpMachine();
            Message($"Result: {r}");
        }
        Reset(q);
    }
}

Main()


Running multiple shots for an expression. Each shot uses an independent instance of the simulator. A list of results (or runtime errors) is returned.

In [ ]:
%%qsharp

operation RandomBit() : Result {
    use q = Qubit();
    H(q);
    let res = M(q);
    Reset(q);
    return res;
}


In [ ]:
results = qsharp.run("RandomBit()", 10)

results


The results can then be processed, e.g. plotted in a histogram using popular Python libraries.

In [ ]:
%pip install matplotlib


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Sort the results so that the histogram labels appear in the correct order
results.sort()
# Count the number of times each result appears
counts = Counter(results)

(values, counts) = counts.keys(), counts.values()
xlabels = np.arange(len(counts))
plt.bar(xlabels, counts)
plt.xticks(xlabels, values)
plt.show()


In [ ]:
%pip install pandas


In [ ]:
import pandas
from collections import Counter

# Sort the results so that the histogram labels appear in the correct order
results.sort()
pandas.Series(results).value_counts(sort=False).plot(kind='bar')


A compiler error in the entry expression:

In [ ]:
qsharp.run("""RandomBit("a")""", 10)


Some shots throw runtime errors:

In [ ]:
%%qsharp

operation Bad() : Unit {
    use q = Qubit();
    H(q);
    let res = M(q);
    if (res == One) {
        // Do something bad, sometimes
        use q2 = Qubit();
        X(q2);
    }
}


In [ ]:
qsharp.run("Bad()", 10)
